<a href="https://colab.research.google.com/github/rakosdonja/product-category-classifier/blob/main/notebooks/02_product_category_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Product Category Classification – Modeling

This notebook focuses on building and evaluating machine learning models for automatic product category prediction based on product titles.

In this phase, we:
- define features and target variables,
- split the data into training and test sets,
- apply appropriate text and numeric preprocessing,
- train multiple machine learning models,
- evaluate their performance and select the best-performing approach.

This notebook builds directly on the cleaned and prepared dataset from the EDA phase.


## Load dataset from GitHub

In this step, we load the `products.csv` file directly from the GitHub repository using its raw URL.
This ensures the notebook is reproducible (no manual uploads) and everyone on the team works with the same dataset version.


In [1]:
import pandas as pd

# Load dataset from GitHub (raw link)
url = "https://raw.githubusercontent.com/rakosdonja/product-category-classifier/main/data/products.csv"
df = pd.read_csv(url)

print("Shape (rows, cols):", df.shape)
display(df.head())


Shape (rows, cols): (35311, 8)


,product ID,Product Title,Merchant ID,Category Label,_Product Code,Number_of_Views,Merchant Rating,Listing Date
0,1,apple iphone 8 plus 64gb silver,1,Mobile Phones,QA-2276-XC,860.0,2.5,5/10/2024
1,2,apple iphone 8 plus 64 gb spacegrau,2,Mobile Phones,KA-2501-QO,3772.0,4.8,12/31/2024
2,3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,3,Mobile Phones,FP-8086-IE,3092.0,3.9,11/10/2024
3,4,apple iphone 8 plus 64gb space grey,4,Mobile Phones,YI-0086-US,466.0,3.4,5/2/2022
4,5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,5,Mobile Phones,NZ-3586-WP,4426.0,1.6,4/12/2023
